In [ ]:
# Import necessary libraries
import numpy as np
import matplotlib.pyplot as plt
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from skimage import transform, filters, exposure
from skimage.util import random_noise
import cv2

# Set style and random seed
plt.style.use('seaborn-v0_8')
np.random.seed(42)

print("Libraries imported successfully!")


In [ ]:
# Create synthetic images for demonstration
def create_sample_images():
    """Create sample images for augmentation demonstration"""
    
    # Create synthetic images with different patterns
    img_size = 128
    images = []
    labels = []
    
    # Pattern 1: Circles
    img1 = np.zeros((img_size, img_size, 3))
    center = img_size // 2
    for radius in [20, 30, 40]:
        y, x = np.ogrid[:img_size, :img_size]
        mask = ((x - center)**2 + (y - center)**2) <= radius**2
        img1[mask] = [1.0, 0.5, 0.0]  # Orange circles
    
    # Pattern 2: Stripes
    img2 = np.zeros((img_size, img_size, 3))
    for i in range(0, img_size, 15):
        img2[i:i+7, :] = [0.0, 0.8, 0.2]  # Green stripes
    
    # Pattern 3: Checkerboard
    img3 = np.zeros((img_size, img_size, 3))
    square_size = 16
    for i in range(0, img_size, square_size):
        for j in range(0, img_size, square_size):
            if (i // square_size + j // square_size) % 2 == 0:
                img3[i:i+square_size, j:j+square_size] = [0.2, 0.4, 0.8]  # Blue squares
    
    images = [img1, img2, img3]
    labels = ['Circles', 'Stripes', 'Checkerboard']
    
    return np.array(images), labels

# Create sample dataset
sample_images, image_labels = create_sample_images()

# Display original images
fig, axes = plt.subplots(1, 3, figsize=(15, 5))
for i, (img, label) in enumerate(zip(sample_images, image_labels)):
    axes[i].imshow(img)
    axes[i].set_title(f'Original: {label}')
    axes[i].axis('off')

plt.tight_layout()
plt.show()

print(f"Created {len(sample_images)} sample images with shape {sample_images[0].shape}")


In [ ]:
# Implement comprehensive augmentation functions
class DataAugmentationSuite:
    """Comprehensive data augmentation implementation"""
    
    def __init__(self, random_state=42):
        self.random_state = random_state
        np.random.seed(random_state)
    
    def geometric_augmentations(self, image):
        """Apply geometric transformations"""
        augmented = {}
        
        # Rotation
        angle = np.random.uniform(-30, 30)
        augmented['rotation'] = transform.rotate(image, angle, mode='wrap', preserve_range=True)
        
        # Scaling
        scale = np.random.uniform(0.8, 1.2)
        augmented['scale'] = transform.rescale(image, scale, mode='wrap', preserve_range=True, multichannel=True)
        
        # Translation
        shift_x = np.random.uniform(-0.1, 0.1) * image.shape[1]
        shift_y = np.random.uniform(-0.1, 0.1) * image.shape[0]
        tform = transform.AffineTransform(translation=(shift_x, shift_y))
        augmented['translation'] = transform.warp(image, tform, mode='wrap', preserve_range=True)
        
        # Horizontal flip
        augmented['h_flip'] = np.fliplr(image)
        
        # Vertical flip
        augmented['v_flip'] = np.flipud(image)
        
        # Shear
        shear = np.random.uniform(-0.2, 0.2)
        tform = transform.AffineTransform(shear=shear)
        augmented['shear'] = transform.warp(image, tform, mode='wrap', preserve_range=True)
        
        return augmented
    
    def color_augmentations(self, image):
        """Apply color space transformations"""
        augmented = {}
        
        # Brightness adjustment
        brightness_factor = np.random.uniform(0.7, 1.3)
        augmented['brightness'] = np.clip(image * brightness_factor, 0, 1)
        
        # Contrast adjustment
        contrast_factor = np.random.uniform(0.7, 1.3)
        mean = np.mean(image)
        augmented['contrast'] = np.clip((image - mean) * contrast_factor + mean, 0, 1)
        
        # Saturation adjustment (for RGB images)
        hsv = cv2.cvtColor((image * 255).astype(np.uint8), cv2.COLOR_RGB2HSV).astype(np.float32)
        hsv[:, :, 1] *= np.random.uniform(0.7, 1.3)  # Saturation channel
        hsv = np.clip(hsv, 0, 255)
        augmented['saturation'] = cv2.cvtColor(hsv.astype(np.uint8), cv2.COLOR_HSV2RGB) / 255.0
        
        # Hue shift
        hsv = cv2.cvtColor((image * 255).astype(np.uint8), cv2.COLOR_RGB2HSV).astype(np.float32)
        hsv[:, :, 0] += np.random.uniform(-20, 20)  # Hue channel
        hsv[:, :, 0] = np.clip(hsv[:, :, 0], 0, 179)
        augmented['hue'] = cv2.cvtColor(hsv.astype(np.uint8), cv2.COLOR_HSV2RGB) / 255.0
        
        # Gamma correction
        gamma = np.random.uniform(0.7, 1.3)
        augmented['gamma'] = np.power(image, gamma)
        
        return augmented
    
    def noise_augmentations(self, image):
        """Apply noise-based augmentations"""
        augmented = {}
        
        # Gaussian noise
        noise_var = np.random.uniform(0.0, 0.05)
        augmented['gaussian_noise'] = random_noise(image, mode='gaussian', var=noise_var, clip=True)
        
        # Salt and pepper noise
        augmented['salt_pepper'] = random_noise(image, mode='s&p', amount=0.05, clip=True)
        
        # Speckle noise
        augmented['speckle'] = random_noise(image, mode='speckle', var=0.05, clip=True)
        
        # Blur
        sigma = np.random.uniform(0.5, 2.0)
        augmented['blur'] = filters.gaussian(image, sigma=sigma, multichannel=True)
        
        return augmented
    
    def advanced_augmentations(self, image1, image2=None, alpha=0.5):
        """Apply advanced augmentation techniques"""
        augmented = {}
        
        # Cutout (random erasing)
        cutout_img = image1.copy()
        h, w = cutout_img.shape[:2]
        cut_h = int(h * 0.2)  # 20% of height
        cut_w = int(w * 0.2)  # 20% of width
        
        # Random position
        y = np.random.randint(0, h - cut_h)
        x = np.random.randint(0, w - cut_w)
        cutout_img[y:y+cut_h, x:x+cut_w] = 0  # Black cutout
        augmented['cutout'] = cutout_img
        
        # Random erasing with random color
        erase_img = image1.copy()
        erase_color = np.random.rand(3)
        erase_img[y:y+cut_h, x:x+cut_w] = erase_color
        augmented['random_erase'] = erase_img
        
        # Mixup (if second image provided)
        if image2 is not None:
            augmented['mixup'] = alpha * image1 + (1 - alpha) * image2
        
        return augmented

# Demonstrate augmentations
augmenter = DataAugmentationSuite()

# Apply augmentations to first image
test_image = sample_images[0]

# Geometric augmentations
geo_augs = augmenter.geometric_augmentations(test_image)

fig, axes = plt.subplots(2, 3, figsize=(18, 12))
axes = axes.flatten()

for i, (name, aug_img) in enumerate(geo_augs.items()):
    axes[i].imshow(aug_img)
    axes[i].set_title(f'Geometric: {name.title()}')
    axes[i].axis('off')

plt.suptitle('Geometric Augmentations', fontsize=16)
plt.tight_layout()
plt.show()

# Color augmentations
color_augs = augmenter.color_augmentations(test_image)

fig, axes = plt.subplots(2, 3, figsize=(18, 12))
axes = axes.flatten()

for i, (name, aug_img) in enumerate(color_augs.items()):
    axes[i].imshow(np.clip(aug_img, 0, 1))
    axes[i].set_title(f'Color: {name.title()}')
    axes[i].axis('off')

plt.suptitle('Color Space Augmentations', fontsize=16)
plt.tight_layout()
plt.show()

# Noise augmentations
noise_augs = augmenter.noise_augmentations(test_image)

fig, axes = plt.subplots(2, 2, figsize=(12, 12))
axes = axes.flatten()

for i, (name, aug_img) in enumerate(noise_augs.items()):
    axes[i].imshow(np.clip(aug_img, 0, 1))
    axes[i].set_title(f'Noise: {name.title()}')
    axes[i].axis('off')

plt.suptitle('Noise-Based Augmentations', fontsize=16)
plt.tight_layout()
plt.show()

# Advanced augmentations
advanced_augs = augmenter.advanced_augmentations(test_image, sample_images[1])

fig, axes = plt.subplots(1, 3, figsize=(18, 6))

for i, (name, aug_img) in enumerate(advanced_augs.items()):
    axes[i].imshow(np.clip(aug_img, 0, 1))
    axes[i].set_title(f'Advanced: {name.title()}')
    axes[i].axis('off')

plt.suptitle('Advanced Augmentations', fontsize=16)
plt.tight_layout()
plt.show()

print("Augmentation demonstrations completed!")
